In [81]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# Veri setini yükleme
file_path = "./Kitap1.xlsx"
data = pd.read_excel(file_path)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Veri setini yükleme
file_path = "./Kitap1.xlsx"
data = pd.read_excel(file_path)

X = data.drop(columns=['Fiyat'])
y = data['Fiyat']
X_filled = X.fillna(X.mean())

X_train_filled, X_test_filled, y_train_filled, y_test_filled = train_test_split(X_filled, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled_filled = scaler.fit_transform(X_train_filled)
X_test_scaled_filled = scaler.transform(X_test_filled)

In [85]:
y_train_filled.shape

(1431,)

In [89]:
# Verileri tensorlere dönüştür
X_train_tensor = torch.tensor(X_train_scaled_filled, dtype=torch.float)
y_train_tensor = torch.tensor(y_train_filled, dtype=torch.float).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled_filled, dtype=torch.float)
y_test_tensor = torch.tensor(y_test_filled.to_numpy(), dtype=torch.float).view(-1, 1)

# Lineer regresyon modeli
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.linear(x)

# Model ve optimizer
model = LinearRegressionModel(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Eğitim döngüsü
model.train()
for epoch in range(20):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Model değerlendirme
model.eval()
with torch.no_grad():
    predicted = model(X_test_tensor)
    test_mse = criterion(predicted, y_test_tensor)
    print(f'Test MSE: {test_mse.item()}')

Epoch 0, Loss: 0.9843590259552002
Epoch 10, Loss: 0.659024715423584
Test MSE: 0.5214120149612427


### GNN Model

In [76]:
# Veriyi [0, 1] aralığında ölçeklendir
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_filled.to_numpy())

# Eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y.to_numpy(), test_size=0.2, random_state=42)

# Eğitim ve test grafiklerini oluştur
train_edge_index = torch.combinations(torch.arange(X_train.shape[0]), r=2).t().contiguous()
train_graph = Data(x=torch.tensor(X_train, dtype=torch.float), edge_index=train_edge_index, y=torch.tensor(y_train, dtype=torch.float))
test_edge_index = torch.combinations(torch.arange(X_test.shape[0]), r=2).t().contiguous()
test_graph = Data(x=torch.tensor(X_test, dtype=torch.float), edge_index=test_edge_index, y=torch.tensor(y_test, dtype=torch.float))


In [77]:
from torch_geometric.nn import GCNConv, global_mean_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(train_graph.num_node_features, 16)
        self.conv2 = GCNConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x.squeeze()

# Model ve optimizer
model = GNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Eğitim döngüsü
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(train_graph)
    loss = torch.nn.functional.mse_loss(out, train_graph.y)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Model değerlendirme
model.eval()
with torch.no_grad():
    pred = model(test_graph)
    test_mse = torch.nn.functional.mse_loss(pred, test_graph.y)
    print(f'Test MSE: {test_mse.item()}')


Epoch 0, Loss: 0.47541531920433044
Epoch 10, Loss: 0.17180182039737701
Epoch 20, Loss: 0.17037759721279144
Epoch 30, Loss: 0.17014369368553162
Epoch 40, Loss: 0.1701340228319168
Epoch 50, Loss: 0.17011970281600952
Epoch 60, Loss: 0.1699453592300415
Epoch 70, Loss: 0.1696765124797821
Epoch 80, Loss: 0.16946321725845337
Epoch 90, Loss: 0.16932232677936554
Epoch 100, Loss: 0.16918312013149261
Epoch 110, Loss: 0.16904129087924957
Epoch 120, Loss: 0.1689068228006363
Epoch 130, Loss: 0.16877523064613342
Epoch 140, Loss: 0.16864821314811707
Epoch 150, Loss: 0.16852585971355438
Epoch 160, Loss: 0.16840860247612
Epoch 170, Loss: 0.1682966947555542
Epoch 180, Loss: 0.16818967461585999
Epoch 190, Loss: 0.1680881232023239
Test MSE: 0.23478026688098907


In [78]:
class SimplifiedGNN(torch.nn.Module):
    def __init__(self):
        super(SimplifiedGNN, self).__init__()
        self.conv1 = GCNConv(train_graph.num_node_features, 8)  # Katman boyutunu düşürdük
        self.conv2 = GCNConv(8, 1)  # Son katmanı koruyoruz

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x.squeeze()

# Daha düşük bir öğrenme oranı kullanarak modeli yeniden eğitelim
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)  # Öğrenme oranını düşürdük

# Eğitim döngüsü ve test MSE hesaplama kodu aynı kalabilir
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(train_graph)
    loss = torch.nn.functional.mse_loss(out, train_graph.y)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Model değerlendirme
model.eval()
with torch.no_grad():
    pred = model(test_graph)
    test_mse = torch.nn.functional.mse_loss(pred, test_graph.y)
    print(f'Test MSE: {test_mse.item()}')

Epoch 0, Loss: 0.16799215972423553
Epoch 10, Loss: 0.16786210238933563
Epoch 20, Loss: 0.16770723462104797
Epoch 30, Loss: 0.16745679080486298
Epoch 40, Loss: 0.16734327375888824
Epoch 50, Loss: 0.16724544763565063
Epoch 60, Loss: 0.16717678308486938
Epoch 70, Loss: 0.16711856424808502
Epoch 80, Loss: 0.16707788407802582
Epoch 90, Loss: 0.16704633831977844
Epoch 100, Loss: 0.16702060401439667
Epoch 110, Loss: 0.16699731349945068
Epoch 120, Loss: 0.16697651147842407
Epoch 130, Loss: 0.16695629060268402
Epoch 140, Loss: 0.16693583130836487
Epoch 150, Loss: 0.1669151782989502
Epoch 160, Loss: 0.16689427196979523
Epoch 170, Loss: 0.16687306761741638
Epoch 180, Loss: 0.16685156524181366
Epoch 190, Loss: 0.16682961583137512
Test MSE: 0.23216639459133148
